# Run all the experiments for the MRMR paper

Performs all the experiments. 

In [ ]:
import sys
sys.path.append("../..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
from pprint import pprint

from wbf_simulate import run_nrobot1day

ModuleNotFoundError: No module named 'exp_run_config'

In [ ]:
experiment = "mrmr"
runs = ["mrmr-clustered-1pio-2con", "mrmr-clustered-3lawnmowers",
        "mrmr-clustered-3random", "mrmr-unclustered-1pio-2con","mrmr-unclustered-3lawnmowers", "mrmr-unclustered-3random"]

In [ ]:
for run in runs:
    exp = Config().get_experiment(experiment, run)
    pprint(exp)
    run_nrobot1day(exp)

['mrmr-clustered-1pio-2con', 'mrmr-clustered-3lawnmowers', 'mrmr-clustered-3random', 'mrmr-unclustered-1pio-2con', 'mrmr-unclustered-3lawnmowers', 'mrmr-unclustered-3random']
